Uso de Pandas y Seaborn + algún gráfico de Plotly.

Dataset: diamonds.csv

* Carga con Pandas: pd.read_csv

* Limpieza de nulos (limpiar valores NaN):
    * Nulos en columnas continuas: mediana, media
    * Nulos en columnas categóricas: moda, un valor fijo

* Cambio de tipo de dato: .astype() por ejemplo a categorical

* Limpieza de valores error: 
    * hay columnas que tienen un valor '?', por tanto se deben reemplazar por una mediana o moda o valor fijo.

* Encoding: texto a numérico
    * Uso de la función get_dummies() para encoding one_hot
    * Uso de map para encoding ordinal para la columna cut: 1, 2, 3, 4

* Uso de función apply:
    * crear una columna price_iva a partir de la columna price que muestre el precio + IVA (21%)

* Crear una nueva columna volumen combinando: x, y, z

* Ordenar por dos columnas con sort_values():
    * tipo de corte (cut) y precio (price)

* Agrupaciones con groupby y visualizarla
    * Agrupar por las 3 que hay de tipo categórico calculando la media, max, min por ejemplo de alguna de las numéricas: price, carat, depth

* Seaborn EDAS:
    * univariantes:
        * histogramas y curvas de densidad
        * boxplot
        * countplot
    * bivariantes y multivariantes
        * scatterplot con hue, con size, con style
        * Calcular correlación con Pandas y mostrarla con seaborn
        * Hacer la correlación en un gráfico de barras para la columna 'price'
    * Combinarlas con:
        * hue, style
        * filtro

* Outliers: Visualización Q1 y Q3 y calcular límites tukey y filtrar. Sobre la columna precio.

* asimetría, curtosis y transformar datos con logaritmo o raíz cuadrada

* Discretizar la columna precio por barato, medio, caro usando la función pd.cut

### Librerías a utilizar

In [37]:
import pandas as pd
import seaborn as sns

### Carga csv con pandas

In [ ]:
df = pd.read_csv("diamonds.csv")
df.head(2)

In [ ]:
df.info()


In [ ]:
#df["price"] = df["price"].astype("float")
#mean()

### Limpieza NaN

In [ ]:
df.count()

In [ ]:
df.describe(include="all").T

In [ ]:
df.isna().sum()

* Para el reemplazo de NaN se utilizará el siguiente critero:
    - carat     media
    - cut       "other"
    - color     "other"
    - clarity   "other"
    - table     media
    - price     "other"
    - x         "other"
    - y         media
    - z         media

In [74]:
"""def nan_media(var):
    media = df[var].mean()
    
    
def nan_text(var, oth):
    df[var] = df[var].fillna(oth)"""

In [83]:
df["carat"] = df["carat"].fillna(df["carat"].mean())
df["cut"] = df["cut"].ffill()
df["color"] = df["color"].fillna(method="ffill")
df["clarity"] = df["clarity"].fillna(method="ffill")
df["table"] = df["table"].fillna(df["table"].mean())
df["price"] = df["price"].fillna(method="ffill")
df["x"] = df["x"].fillna(method="ffill")
df["y"] = df["y"].fillna(df["y"].mean())
df["z"] = df["z"].fillna(df["z"].mean())

C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["color"] = df["color"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["clarity"] = df["clarity"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["price"] = df["price"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["x"] = df["x"].fillna(method="ffill")


In [82]:
df.count()


carat      53940
cut        53940
color      53940
clarity    53940
depth      53940
table      53940
price      53940
x          53940
y          53940
z          53940
dtype: int64

### Limpieza valores error

In [48]:
#df.drop()

### Cambio de tipo de datos

In [49]:
#

## Encoding

* Get_dummies()

In [50]:
#

* Encodig Ordinal

In [51]:
#

### Creacion columnas

* price_iva

In [52]:
#

* volumen ( x, y, z)

In [53]:
#

### Orden de columnas


* Tipo de corte

* Precio

### Agrupaciones group_by()

* carac

In [54]:
#

* price

In [55]:
#

* depth

In [56]:
#

## Seaborn EDAS
### Univariantes

* Histograma y curva de densidad

In [57]:
#

* Boxplot

In [58]:
#

* Countplot

In [59]:
#

### Bivariantes y Multivariantes
* Scatterplot

In [60]:
# con hue


In [61]:
# con style


In [62]:
# con size

#### Correlacion
* Calculo con pandas

In [63]:
#

* Mostrar con Seaborn

In [64]:
#

* Combinarlas con filtros hue, style

In [65]:
#

## Outliers
* Visualizacion Q1 y Q3 

In [66]:
#

* Limites Tukey y filtro

In [67]:
#

## Transformacion de datos
* Asimetria

* Curtosis

* Transformacion logaritmo

* Transformacion raiz cuadrada

## Discretizar la columna precio
* barato, medio, caro